### In this notebook we show how to implement the btag corrections. 

In [1]:
cd ..

/home/cms-jovyan/wprime_plus_b/analysis


In [2]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

We'll use some $t \bar{t}$ semileptonic data in this example

In [3]:
fname = "root://xcache//store/mc/RunIISummer20UL17NanoAODv2/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/120000/2A2F4EC9-F9BB-DF43-B08D-525B5389937E.root"
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

Let's define our bjets. For 2017 data, the `btagDeepFlavB` is 0.304 for the *Medium* working point (see the `data/btagWPs.json`)

In [4]:
good_bjets = (
    (events.Jet.pt > 30)
    & (events.Jet.jetId == 6)
    & (events.Jet.puId == 7)
    & (events.Jet.btagDeepFlavB > 0.304)
)
bjets = events.Jet[good_bjets]

Let's create an instance of the `BTagCorrector` class

In [5]:
from corrections import BTagCorrector

BTagCorrector?

Init signature:
BTagCorrector(
    sf: str = 'comb',
    wp: str = 'M',
    tagger: str = 'deepJet',
    year: str = '2017',
    mod: str = '',
)
Docstring:      <no docstring>
Init docstring:
BTag corrector object

Parameters:.coffea
-----------
    sf:
        scale factors to use (mujets or comb)
    wp:
        worging point (L, M or T)
    tagger:
        tagger (deepJet or deepCSV)
    year:
        dataset year
    mod:
        year modifier ("" or "APV")
File:           ~/wprime_plus_b/analysis/corrections.py
Type:           type
Subclasses:     


From the [documentation](https://cms-nanoaod-integration.web.cern.ch/commonJSONSFs/BTV_btagging_Run2_UL/BTV_btagging_2017_UL.html):

"For the working point corrections the SFs in 'mujets' and 'comb' are for b/c jets. The 'mujets' SFs contain only corrections derived in QCD-enriched regions. The 'comb' SFs  contain corrections derived in QCD and ttbar-enriched regions. Hence, 'comb' SFs can be used  everywhere, except for ttbar-dileptonic enriched analysis regions. For the ttbar-dileptonic regions the 'mujets' SFs should be used. The 'incl' correction is for light-flavoured jets."

We'll use the `deepJet_comb` corrections:

In [6]:
btag = BTagCorrector(sf="comb", wp="M", tagger="deepJet", year="2017")

The b-tag scale factors can be obtained from the `btagSF` method, which takes in the light-flavoured or heavy-flavoured jets and the systematics ("central" for nominal weights). In our case we use heavy-flavoured jets (b-jets). We can set the flavour using the `hadronFlavour` field (0 = udsg, 4 = c, 5 = b)

In [7]:
bjets = bjets[(bjets.hadronFlavour > 0) & (abs(bjets.eta) < 2.5)]

bSF = btag.btag_SF(bjets, syst="central")
bSF

<Array [[0.967], [0.962, ... [0.973, 0.967]] type='1280000 * var * float64'>

the nominal weight is the product of the event weights

In [8]:
nominal_weight = ak.prod(bSF, axis=-1)
nominal_weight

<Array [0.967, 0.924, 1, ... 0.964, 0.942] type='1280000 * float64'>

This nominal weight can be added to an instance of the `Weights` class, which is the object that allows us to manipulate weights easily and efficiently

In [9]:
from coffea.analysis_tools import Weights

# instance of the Weights class
weights = Weights(len(events), storeIndividual=True)

# add nominal btag weight
weights.add(name="btagSF", weight=nominal_weight)

We can acces to the combination of all weights added to `weights` using the `weight` method

In [10]:
weights.weight()

array([0.96706041, 0.92435078, 1.        , ..., 0.9664197 , 0.96360567,
       0.94176765])

We can access an individual weight or a subset of the weights added to `weights` using the `partial_weight` method, indicating the weights that we want to include in the selection (this is only possible if we set `storeIndividual=True`)

In [11]:
weights.partial_weight(include=["btagSF"])

array([0.96706041, 0.92435078, 1.        , ..., 0.9664197 , 0.96360567,
       0.94176765])

Alternatively, we can use the `add_btag_weight` method to add the btag SF to the weights

In [12]:
btag.add_btag_weight?

Signature:
btag.add_btag_weight(
    jets: awkward.highlevel.Array,
    weights: Type[coffea.analysis_tools.Weights],
)
Docstring:
add b-tagging scale factor

Parameters:
-----------
    jets:
        jets selected in your analysis
    weights:
        Weights object from coffea.analysis_tools
File:      ~/wprime_plus_b/analysis/corrections.py
Type:      method


In [13]:
# bjets
good_bjets = (
    (ak.firsts(events.Jet.pt) > 30)
    & (events.Jet.jetId == 6)
    & (events.Jet.puId == 7)
    & (events.Jet.btagDeepFlavB > 0.304)
)
bjets = events.Jet[good_bjets]

# instance of the Weights class
weights = Weights(len(events), storeIndividual=True)

# add nominal weight to weights
btag = BTagCorrector(sf="comb", wp="M", tagger="deepJet", year="2017")
btag.add_btag_weight(jets=bjets, weights=weights)

In [14]:
weights.partial_weight(include=["btagSF"])

array([0.96706041, 0.92435078, 1.        , ..., 0.9664197 , 0.96360567,
       0.94176765])